## QuickStart with shioaji

In [1]:
! pip install -U shioaji

In [1]:
import shioaji as sj

In [3]:
import json
with open("login.json", "r") as f:
    login_kwargs = json.loads(f.read())

In [7]:
api = sj.Shioaji()

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


In [8]:
accounts = api.login(**login_kwargs, contracts_cb=print)

SecurityType.Index
SecurityType.Future
SecurityType.Option
SecurityType.Stock


In [2]:
api = sj.Shioaji(simulation=True)

Response Code: 0 | Event Code: 0 | Info: host '218.32.76.102:80', IP 218.32.76.102:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


### testing account
PAPIUSER01
...
PAPIUSER10

In [3]:
api.login("PAPIUSER02", "2222", contracts_cb=print)

SecurityType.Index
SecurityType.Future


[FutureAccount(person_id='PAPIUSER02', broker_id='F002000', account_id='9100295', signed=True, username='PAPIUSER02'),
 StockAccount(person_id='PAPIUSER02', broker_id='9A95', account_id='0504486', signed=True, username='PAPIUSER02')]

SecurityType.Option
SecurityType.Stock


### shioaji there main components
- Contract
- Order
- Trade

In [9]:
api.Contracts

Contracts(Indexs=(OTC, TSE), Stocks=(OES, OTC, TSE), Futures=(BRF, BTF, CAF, CBF, CCF, CDF, CEF, CFF, CGF, CHF, CJF, CKF, CLF, CMF, CNF, CQF, CRF, CSF, CUF, CWF, CXF, CYF, CZF, DCF, DD1, DDF, DEF, DFF, DGF, DHF, DIF, DJF, DKF, DLF, DNF, DOF, DPF, DQF, DSF, DUF, DVF, DWF, DXF, DYF, DZF, E4F, EEF, EGF, EHF, EMF, EPF, ERF, EXF, EYF, FEF, FFF, FGF, FKF, FQF, FRF, FTF, FVF, FWF, FXF, FYF, FZF, G2F, GAF, GCF, GDF, GHF, GIF, GJF, GLF, GMF, GNF, GOF, GRF, GTF, GUF, GWF, GXF, GZF, HAF, HBF, HCF, HHF, HIF, HLF, HOF, HSF, HYF, IAF, IHF, IIF, IJF, IMF, IOF, IPF, IQF, IRF, ITF, IXF, IYF, IZF, JBF, JFF, JNF, JPF, JSF, JWF, JZF, KAF, KB1, KBF, KCF, KDF, KEF, KFF, KGF, KIF, KKF, KLF, KOF, KPF, KSF, KWF, LBF, LCF, LEF, LIF, LMF, LOF, LQF, LRF, LTF, LUF, LV1, LVF, LWF, LXF, LYF, MAF, MBF, MCF, MJF, MKF, MPF, MQF, MVF, MX4, MXF, MYF, NAF, NBF, NCF, NDF, NEF, NGF, NIF, NJF, NLF, NMF, NOF, NQF, NSF, NUF, NVF, NWF, NXF, NYF, NZF, OAF, OBF, OCF, ODF, OEF, OGF, OHF, OJF, OKF, OLF, OMF, OOF, OPF, OQF, ORF, OSF

In [11]:
stock_contract = api.Contracts.Stocks["2330"]
stock_contract

Stock(exchange=<Exchange.TSE: 'TSE'>, code='2330', symbol='TSE2330', name='台積電', category='24', limit_up=343.0, limit_down=281.0, reference=312.0, update_date='2020/06/23', margin_trading_balance=2990, short_selling_balance=177, day_trade=<DayTrade.Yes: 'Yes'>)

In [13]:
api.Contracts.Futures.TXF

TXF(TXF202106, TXF202009, TXF202008, TXF202007, TXF202012, TXF202103)

In [15]:
futures_contract = api.Contracts.Futures.TXF.TXF202007
futures_contract

Future(code='TXFG0', symbol='TXF202007', name='臺股期貨', category='TXF', delivery_month='202007', underlying_kind='I', limit_up=12563.0, limit_down=10279.0, reference=11421.0, update_date='2020/06/23')

### Subscribe Streaming Quote Data

In [16]:
api.quote.subscribe(api.Contracts.Stocks["2330"], sj.constant.QuoteType.Tick)

Response Code: 200 | Event Code: 16 | Info: MKT/*/TSE/2330 | Event: Subscribe or Unsubscribe ok


In [19]:
api.quote.unsubscribe(api.Contracts.Futures.TXF.TXF202007)

Response Code: 200 | Event Code: 16 | Info: L/*/TXFG0 | Event: Subscribe or Unsubscribe ok


In [17]:
@api.quote.on_quote
def on_quote(topic: str, quote: dict):
    print(f"Topic: {topic}, Quote: {quote}")

In [18]:
api.quote.subscribe(api.Contracts.Futures.TXF.TXF202007)

Response Code: 200 | Event Code: 16 | Info: L/*/TXFG0 | Event: Subscribe or Unsubscribe ok
Topic: L/TFE/TXFG0, Quote: {'Amount': [11494.0], 'AmountSum': [66735843.0], 'AvgPrice': [11486.37573149742], 'Close': [11494.0], 'Code': 'TXFG0', 'Date': '2020/06/23', 'DiffPrice': [38.0], 'DiffRate': [0.33170391061452514], 'DiffType': [2], 'High': [11500.0], 'Low': [11460.0], 'Open': 11470.0, 'TargetKindPrice': 11612.36, 'TickType': [2], 'Time': '16:03:37.908000', 'TradeAskVolSum': 3621, 'TradeBidVolSum': 2957, 'VolSum': [5810], 'Volume': [1]}


### Placing Order

In [20]:
sj.Order?

Init signature:
sj.Order(
    price: Union[pydantic.types.StrictInt, int],
    quantity: shioaji.order.ConstrainedIntValue,
    action: shioaji.constant.Action,
    price_type: Union[shioaji.constant.StockPriceType, shioaji.constant.FuturesPriceType, shioaji.constant.TFTStockPriceType],
    order_type: Union[shioaji.constant.StockOrderType, shioaji.constant.FuturesOrderType, shioaji.constant.TFTOrderType],
    **kwargs,
)
Docstring:      <no docstring>
File:           ~/.pyenv/versions/miniconda3-latest/lib/python3.7/site-packages/shioaji/order.py
Type:           ModelMetaclass
Subclasses:     


In [23]:
sj.order.StockOrder??

Init signature: sj.order.StockOrder(**data: Any) -> None
Docstring:      <no docstring>
Source:        
class StockOrder(BaseOrder):
    price_type: StockPriceType
    order_type: StockOrderType
    order_cond: StockOrderCond = StockOrderCond.Cash
    first_sell: StockFirstSell = StockFirstSell.No
File:           ~/.pyenv/versions/miniconda3-latest/lib/python3.7/site-packages/shioaji/order.py
Type:           ModelMetaclass
Subclasses:     Order


### more detail inside [shioaji doc](https://sinotrade.github.io/tutor/order/Stock/)

### Currently, focus on streaming data processing

build the class for special handling realtime quote ( use class can access the member variables replace the single function with global variables to obtain more testability )

the most simple example use redis as data collector just collect data into it and handle it later

In [7]:
import redis
import msgpack
import pandas as pd

class RedisQuoteManager:
    def __init__(self, rs: redis.Redis):
        self.rs = rs
    
    def on_quote(self, topic: str, quote: dict):
        self.rs.rpush(topic, msgpack.dumps(quote))

In [2]:
from sjddb.quote_managers import flatten_tick

In [3]:
rs = redis.Redis(db=0)
rqm = RedisQuoteManager(rs)
api.quote.set_quote_callback(rqm.on_quote)

In [12]:
data = [msgpack.loads(r, raw=False) for r in rs.lrange(rs.keys("MKT*/2330")[0], 0, -2)]

In [23]:
df = pd.DataFrame([flatten_tick(d, i) for d in data for i, _ in enumerate(d["Close"])])

In [24]:
df["Datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"], format="%Y/%m/%d %H:%M:%S.%f")

In [28]:
df = df[["Datetime", "Close", "Volume", "index", "TickType", "Simtrade"]].copy()

In [31]:
df_tick_2330 = df[df['Simtrade'] != 1]

In [42]:
df_tick_2330.head()

,Datetime,Close,Volume,index,TickType,Simtrade
355,2020-06-23 09:00:00.963777,316.0,2586,0,1,NaN
356,2020-06-23 09:00:00.986819,316.0,1,0,2,NaN
357,2020-06-23 09:00:01.084297,316.0,10,0,2,NaN
358,2020-06-23 09:00:01.170694,316.0,2,0,2,NaN
359,2020-06-23 09:00:01.588050,316.0,2,0,2,NaN


In [41]:
df_tick_2330.set_index("Datetime").resample('1min')["Close"].ohlc().join(
    df_tick_2330.set_index("Datetime").resample('1min')[["Volume"]].sum()
)

,open,high,low,close,Volume
Datetime,,,,,
2020-06-23 09:00:00,316.0,316.5,315.5,316.0,3162
2020-06-23 09:01:00,315.5,316.0,315.5,315.5,94
2020-06-23 09:02:00,315.5,316.0,315.5,316.0,150
2020-06-23 09:03:00,316.0,316.0,315.5,316.0,664
2020-06-23 09:04:00,316.0,316.0,315.5,316.0,591
2020-06-23 09:05:00,316.0,316.0,315.5,316.0,251
2020-06-23 09:06:00,316.0,316.0,315.5,316.0,117
2020-06-23 09:07:00,315.5,316.0,315.5,315.5,72
2020-06-23 09:08:00,315.5,316.0,315.5,315.5,101


### How about multi symbol resample and aggregation?
### Why not use QuoteManager and put all the trading algo into it?